In [69]:
#Libraries 
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.keras
from PIL import Image, ImageOps
from keras.preprocessing import image
import warnings
import gc
from itertools import permutations 
warnings.filterwarnings("ignore")

In [70]:
img_path = r"D:\working repos\AltRekruit\Testing\Final (15jan)\images\good_2.jpg"
scores_1 = {}

**Score Calculating**
- in functions to run mem load out of scope ( temporary implentation ) 

In [71]:
def vgg_19(img_path):
    global scores
    vgg19 = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\vgg19.h5')
    
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    vgg19_score = vgg19.predict(img)
    
    scores_1['vgg19'] = vgg19_score

    
def vgg_16(img_path):
    global scores
    vgg16 = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\vgg16.h5')
    
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    vgg16_score = vgg16.predict(img)
    
    scores_1['vgg16'] = vgg16_score
    

def resnet_50(img_path):
    global scores
    resnet50 = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\res50.h5')
    
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    resnet50_score = resnet50.predict(img)
    
    scores_1['resnet50'] = resnet50_score
    
    
def naive_model(img_path):
    global scores
    naive = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\naive.h5')
    
    img = image.load_img(img_path, target_size=(64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    naive_score = naive.predict(img)
    
    scores_1['naive'] = naive_score
    
    
def tm_model(img_path):
    tm = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\tm_custom_model(image-pairs).h5')
    
    np.set_printoptions(suppress=True)
    
    tm.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
              metrics=['accuracy'])


    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    img = Image.open(img_path)
    size = (224, 224)
    img = ImageOps.fit(img, size, Image.ANTIALIAS)
    image_array = np.asarray(img)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array

    tm_score = tm.predict(data)
    
    scores_1['tm_custom'] = tm_score
    

gc.collect()

0

In [72]:
vgg_19(img_path) ; gc.collect()
vgg_16(img_path) ; gc.collect()
resnet_50(img_path) ; gc.collect()
naive_model(img_path) ; gc.collect()
tm_model(img_path) ; gc.collect()

1499

In [73]:
val_ = []
cls_t = [] 
# 0 , 1 ,2 Where 2 means equal scores
for key in scores_1.keys():
    if scores_1[key][0][0]>scores_1[key][0][1]:
        cls_t.append(0)
        val_.append(0)
    elif scores_1[key][0][0]<scores_1[key][0][1]:
        val_.append(1)
        cls_t.append(1)
    else:
        val_append(2)


score_1_avg = 0
iter_ = 0 
for key in scores_1.keys():
    act_val = val_[iter_]
    score_1_avg = score_1_avg +  scores_1[key][0][act_val]
gc.collect()
score_1_final = (score_1_avg/5)*100

gc.collect()

0

Second Implementation

In [74]:
def class_finder(vgg_19,naive,btg_res,btb_vgg16,tm):
    global vgg19_cls,naive_cls,tm_cls
    
    if vgg_19[0]>vgg_19[1]:
        vgg19_cls = 0
    else:
        vgg19_cls = 1
    
        
    if naive[0]>naive[1]:
        naive_cls = 0
    else:
        naive_cls = 1
    
    if vgg19_cls == naive_cls:
        
        if vgg19_cls == 0:
            add_on = btb_vgg16
        else:
            add_on = btg_res
        return(vgg_19[vgg19_cls]+naive[vgg19_cls]+add_on[vgg19_cls])/3
    
    else:
        if tm[0]>tm[1]:
            tm_cls = 0
            add_on = btb_vgg16
        else:
            tm_cls = 1
            add_on = btg_res
            
        return(tm[tm_cls]+add_on[tm_cls])/2
        
def most_(List): 
    return max(set(List), key = List.count)
def Average(lst): 
    return sum(lst) / len(lst)

gc.collect()        

20

In [75]:
#g_scores 
score_2_final_g1 = class_finder(scores_1['vgg19'][0],scores_1['resnet50'][0],scores_1['naive'][0],scores_1['vgg16'][0],scores_1['tm_custom'][0])*100
score_2_final_g2 = class_finder(scores_1['vgg19'][0],scores_1['resnet50'][0],scores_1['tm_custom'][0],scores_1['vgg16'][0],scores_1['naive'][0])*100

#b_scores
score_2_final_b1 = class_finder(scores_1['vgg19'][0],scores_1['resnet50'][0],scores_1['naive'][0],scores_1['vgg16'][0],scores_1['tm_custom'][0])*100
score_2_final_b2 = class_finder(scores_1['vgg19'][0],scores_1['resnet50'][0],scores_1['naive'][0],scores_1['tm_custom'][0],scores_1['vgg16'][0])*100



cls_list = [[tm_cls,naive_cls,vgg19_cls],cls_t]
cls_list_n = [item for sublist in cls_list for item in sublist]
count_0 = cls_list_n.count(0)
count_1 = cls_list_n.count(1)

gc.collect()

20

In [76]:
count_0,count_1

(5, 3)

In [77]:
perm = permutations([score_2_final_g1,score_2_final_g2,score_2_final_b1,score_2_final_b2])
g_score = []
b_score = []
for lst in list(perm):
    g_score.append(max(lst))
    b_score.append(min(lst))
g_score = Average(g_score) 
b_score = Average(b_score)
score_2_final = (b_score*count_0 + g_score*count_1)/(count_0+count_1)
final_score = (score_1_final + score_2_final)/2

In [78]:
final_score

64.51955095554392